# Target cost strategy

- Trouver l'allocation des échantillons sur chaque niveau de fidélité (eq. 2.39 Mycek & De Lozzo) :
    - $\mathcal{V}_\ell = \mathbb{V}[f_\ell(X)^2 - f_{\ell-1}(X)^2]$
        - à estimer avec un échantillon "pilote" indépendant de taille $n = 10000$
    - $\mathcal{C}_\ell = 2\mathcal{C}_{\ell-1}$
    - Pour 2 niveaux :
        - $\mathcal{V}_0 = \mathbb{V}[f_0(X)^2]$
        - $\mathcal{V}_1 = \mathbb{V}[f_1(X)^2 - f_0(X)^2]$
        - $\mathcal{C}_1 = 1$
        - $\mathcal{C}_0 = 0.5$
    - on se donne un budget $\eta = 100$.
    - dans notre cas, $m=0$

In [ ]:
import numpy as np 

In [ ]:
# Constants 
d = 10


#print(np.random.uniform(-1,1,d))
#v_1= np.array([-0.18268292, -0.63730044,  0.49158538, -0.51712638, -0.90171302,  0.40921266,  0.63791406,  0.70701292, -0.24888679,  0.94455888])
v_1= np.array([-0.00606533, -0.02837768, -0.20481078, -0.05524456,  0.00408442, -0.02378791, -0.11289296, -0.09047946, -0.0828985,   0.01015773])
#v_0 = np.array([-0.00589986, -0.02848255, -0.2058163,  -0.05332501,  0.00356343, -0.02399142, -0.11355733, -0.08986586, -0.08302782,  0.0085727 ])
eps = np.random.normal(0,0.05,d) 
v_0 = v_1 + eps  #vecteur ceof basse fidélité 
print(v_0)

def f_vec(X,v):
    """
    X : (d,n) matrix
    v : d vector 
    """
    return v.T@X

def f_vec_squared(X,v):
    """
    X : (d,n) matrix
    v : d vector 
    """
    return (v.T@X)**2

exp_var = np.sum(np.square(v_1))

print(exp_var)

Première chose à faire : Trouver $\mathcal{V}_0$ et $\mathcal{V}_1$

In [ ]:
tab_V_0 = []
tab_V_1 = []
tab_Var_1 = []

N = 1000
n0 = 100
n1 = 10

# for _ in range(N):
#     X1 = np.random.standard_normal((d,10000))
#     X0 = np.random.standard_normal((d,))
#     tab_V_0.append(f_vec_squared(X0,v_0))
#     tab_V_1.append(f_vec_squared(X1,v_1) - f_vec_squared(X1,v_0))
#     tab_Var_1.append(f_vec_squared(X0,v_1))



In [ ]:
N = 10000
X = np.random.standard_normal((d,N))
V_0 = np.var(f_vec_squared(X,v_0))
V_1 = np.var(f_vec_squared(X,v_1) - f_vec_squared(X,v_0))

In [ ]:
# V_0 = np.var(tab_V_0)
# V_1 = np.var(tab_V_1)
# Var_1 = np.var(tab_Var_1)

In [ ]:
print(V_0, V_1)
# print(Var_1)

Calcul de $M_\ell = m + 1 + \lfloor (\tilde{\eta} / \mathcal{S}_L) \sqrt{\mathcal{V}_\ell / \mathcal{C}_\ell} \rfloor > m $

avec : 
$$
 \tilde{\eta} = \eta - m \sum_{\ell' \le L} \mathcal{C}_{\ell'} \\
 \mathcal{S}_L = \sum_{\ell \le L} \sqrt{\mathcal{V}_\ell \mathcal{C}_\ell}
 $$


Ici, nous avons :
$$
m = 0 \\
\mathcal{C}_1 = 1 \\
\mathcal{C}_0 = 0.5 \\
\eta = 100
$$

In [ ]:
def M_ell(eta, V_ell, C_ell,ell, m=0):
    """
    eta : budget
    V_ell : tableau contenant [V_0,V_1,...,V_L]
    C_ell : tableau contenant [C_0,C_1,...,C_L]

    return : la formule M_ell au dessus qui nous donne le nombre d'élement optimal à simuler
    """
    eta_tilde = eta # - m*np.sum(...)
    S_L = np.sum(np.sqrt(np.multiply(V_ell,C_ell)))
    print(S_L**2/eta)
    
    return m + 1 + np.floor(
        (eta_tilde/S_L)*np.sqrt(V_ell[ell]/C_ell[ell])
    )


In [ ]:
V_ell = [V_0,V_1]
C_ell = [0.75,1.]
eta=20

M_0 = M_ell(eta,V_ell,C_ell,0)
M_1 = M_ell(eta,V_ell,C_ell,1)

In [ ]:
print(M_0,M_1)

CCL : On aurait 99 pour n_0 et 3 pour n_1, ie 99 simulations basse fidélité et 3 haute fidélité pour obtenir un resultat optimal avec un budget de $\eta = 100$